In [30]:
import torch
import torchvision
from torchvision import transforms, datasets
import os
import cv2
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from skimage import io, transform
from torch import optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader,TensorDataset, Dataset

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
DataDir = "/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/GR/B200_Pictures_GR"

In [33]:
csv_file = "/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/labels_complete_dataset_607.csv"

In [134]:
class GPRDataset(Dataset):
    
    def __init__ (self,root_dir,csv_file, length,  transform):
        self.root_dir = root_dir
        self.transform = transform
        self.length = length
        self.pictures_df = pd.read_csv(csv_file)
       
 
    
    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir,
                                self.pictures_df.iloc[idx, 0])
        image = io.imread(img_name)
        label = self.pictures_df.loc[idx, 'class']
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
    
        return (image, label)
    
    def __len__(self):
        return self.length

In [135]:
img_file = list(os.listdir('/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/GR/B200_Pictures_GR'))
length = len(img_file)

In [136]:
length

668

In [137]:
transform = transforms.Compose(
    [
        transforms.Resize((128,128)),
        transforms.ToTensor()
    ]
)

dataset = GPRDataset(DataDir,csv_file,length, transform = transform)
fig = plt.figure()

<Figure size 432x288 with 0 Axes>

In [138]:
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [139]:
# for i, (image, label) in enumerate(data_loader):
#     print(i)

In [140]:
class GeneratorModel(nn.Module):
    def __init__(self):
        super(GeneratorModel, self).__init__()
        input_dim = 128 + 7
        output_dim = 128
        self.label_embedding = nn.Embedding(7, 7)
        
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )

        self.hidden_layer3 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )

        self.hidden_layer4 = nn.Sequential(
            nn.Linear(1024, output_dim),
            nn.Tanh()
        )
    
    def forward(self, x, labels):
        c = self.label_embedding(labels)
        x = torch.cat([x,c], 1)
        output = self.hidden_layer1(x)
        output = self.hidden_layer2(output)
        output = self.hidden_layer3(output)
        output = self.hidden_layer4(output)
        return output.to(device)

In [141]:
class DiscriminatorModel(nn.Module):
    def __init__(self):
        super(DiscriminatorModel, self).__init__()
        input_dim = 128 + 7
        output_dim = 1
        self.label_embedding = nn.Embedding(7, 7)

        self.hidden_layer1 = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        self.hidden_layer3 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        self.hidden_layer4 = nn.Sequential(
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
#         print('labels ',labels)
        c = self.label_embedding(labels)
        x = torch.cat([x, c], 1)
        output = self.hidden_layer1(x)
        output = self.hidden_layer2(output)
        output = self.hidden_layer3(output)
        output = self.hidden_layer4(output)

        return output.to(device)


In [142]:
discriminator = DiscriminatorModel()
generator = GeneratorModel()
discriminator.to(device)
generator.to(device)

GeneratorModel(
  (label_embedding): Embedding(7, 7)
  (hidden_layer1): Sequential(
    (0): Linear(in_features=135, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer2): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer3): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer4): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): Tanh()
  )
)

In [143]:
loss = nn.BCELoss()
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [144]:
#Training

n_epochs = 32
batch_size = 32
#batch_size = batch_size.type(torch.LongTensor)
for epoch_idx in range(n_epochs):
    G_loss = []
    D_loss = []
    for i, (image, label) in enumerate(data_loader):                
        noise = torch.randn(batch_size,128).to(device)
        fake_labels = torch.randint(0, 7, (batch_size,)).to(device)
        #print(fake_labels.shape)       
        generated_data = generator(noise, fake_labels.type(torch.LongTensor)) # batch_size X 784

        
        # Discriminator
        true_data = image # batch_size X 784
        digit_labels = label# batch_size
        true_labels = torch.ones(batch_size).to(device)
        
        
        discriminator_optimizer.zero_grad()

        discriminator_output_for_true_data = discriminator(true_data, digit_labels).view(batch_size)
        true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)

        discriminator_output_for_generated_data = discriminator(generated_data.detach(), fake_labels).view(batch_size)
        generator_discriminator_loss = loss(
            discriminator_output_for_generated_data, torch.zeros(batch_size).to(device)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2
        
        discriminator_loss.backward()
        discriminator_optimizer.step()

        D_loss.append(discriminator_loss.data.item())
        
        
        # Generator

        generator_optimizer.zero_grad()
        # It's a choice to generate the data again
        generated_data = generator(noise, fake_labels) # batch_size X 784
        discriminator_output_on_generated_data = discriminator(generated_data, fake_labels).view(batch_size)
        generator_loss = loss(discriminator_output_on_generated_data, true_labels)
        generator_loss.backward()
        generator_optimizer.step()
        
        G_loss.append(generator_loss.data.item())
        if ((batch_idx + 1)% 500 == 0 and (epoch_idx + 1)%10 == 0):
            print("Training Steps Completed: ", batch_idx)
            
            with torch.no_grad():
                noise = torch.randn(batch_size,100).to(device)
                fake_labels = torch.randint(0, 10, (batch_size,)).to(device)
                generated_data = generator(noise, fake_labels).cpu().view(batch_size, 128, 128)
                for x in generated_data:
                    print(fake_labels[0].item())
                    plt.imshow(x.detach().numpy(), interpolation='nearest',cmap='gray')
                    plt.show()

                    break


    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch_idx), n_epochs, torch.mean(torch.FloatTensor(D_loss)), torch.mean(torch.FloatTensor(G_loss))))

labels  ('deckschichtschaden', 'mehrfachschichtabloesung', 'mehrfachschichtabloesung', 'schichtabloesung', 'schichtabloesung lang', 'schichtabloesung', 'mehrfachschichtabloesung', 'deckschichtschaden', 'schichtabloesung lang', 'schichtmarker', 'schichtabloesung', 'mehrfachschichtabloesung', 'schichtabloesung', 'schichtabloesung', 'schichtabloesung lang', 'schichtabloesung', 'mehrfachschichtabloesung', 'schichtabloesung', 'mehrfachschichtabloesung', 'deckschichtschaden', 'deckschichtschaden', 'schichtabloesung', 'mehrfachschichtabloesung', 'deckschichtschaden', 'schichtabloesung', 'schichtmarker', 'schichtabloesung lang', 'deckschichtschaden', 'schichtabloesung', 'schichtabloesung', 'schichtabloesung', 'schichtabloesung')


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not tuple